In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [ ]:
image_folder = './gaussian_filtered_images/gaussian_filtered_images'  # Base folder for images
csv_file = './gaussian_filtered_images/gaussian_filtered_images/train.csv'  # CSV file containing image names and labels

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
def load_data(csv_file, image_folder):
    df = pd.read_csv(csv_file)
    image_paths = df['id_code'].values
    labels = df['diagnosis'].values
    
    images = []
    for img_name in image_paths:
        # Determine the subfolder based on the label (0 to 4)
        if labels[df[df['id_code'] == img_name].index[0]] == 0:
            subfolder = 'no_dr'
        elif labels[df[df['id_code'] == img_name].index[0]] == 1:
            subfolder = 'mild'
        elif labels[df[df['id_code'] == img_name].index[0]] == 2:
            subfolder = 'moderate'
        elif labels[df[df['id_code'] == img_name].index[0]] == 3:
            subfolder = 'severe'
        else:
            subfolder = 'proliferate_DR'

        img_path = os.path.join(image_folder, subfolder, f"{img_name}.png")
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)

    images = np.array(images)
    labels = tf.keras.utils.to_categorical(labels, num_classes=5)
    return train_test_split(images, labels, test_size=0.2, random_state=42)

# Load data
train_images, test_images, train_labels, test_labels = load_data(csv_file, image_folder)

# Normalize images
train_images, test_images = train_images / 255.0, test_images / 255.0

# Model definition
input_img = Input(shape=(224, 224, 3))


In [ ]:
train_images, test_images, train_labels, test_labels = load_data(csv_file, image_folder)

# Normalize images
train_images, test_images = train_images / 255.0, test_images / 255.0

# Model definition
input_img = Input(shape=(224, 224, 3))

# Shared layers
x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

In [ ]:
classification_output = Dense(5, activation='softmax', name='classification_output')(x)

# Regression output
regression_output = Dense(1, activation='linear', name='regression_output')(x)

# Model definition
model = Model(inputs=input_img, outputs=[classification_output, regression_output])

# Model compilation
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss={'classification_output': 'categorical_crossentropy', 'regression_output': 'mean_squared_error'},
              metrics={'classification_output': 'accuracy', 'regression_output': 'mae'})


In [ ]:
history = model.fit(train_images, {'classification_output': train_labels, 'regression_output': train_labels[:, 0]},  # Assuming regression targets are correlated with classification labels
                    epochs=20, validation_data=(test_images, {'classification_output': test_labels, 'regression_output': test_labels[:, 0]}))


In [ ]:
evaluation_results = model.evaluate(test_images, 
                                    {'classification_output': test_labels, 'regression_output': test_labels[:, 0]})
print(evaluation_results)


In [ ]:
# Load and preprocess a new image
new_image_path = './val_img.jpg'  # Replace with the actual path to the new image
new_img = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(224, 224))
new_img_array = tf.keras.preprocessing.image.img_to_array(new_img)
new_img_array = new_img_array / 255.0  # Normalize the image
new_img_array = np.expand_dims(new_img_array, axis=0)  # Add batch dimension

# Make predictions
predictions = model.predict(new_img_array)

# Get the classification and regression outputs
classification_prediction = predictions[0]
regression_prediction = predictions[1]

# Convert classification output to class label
predicted_class = np.argmax(classification_prediction)
class_labels = ['no_dr', 'mild', 'moderate', 'severe', 'proliferate_DR']  # Modify these as per your dataset
predicted_label = class_labels[predicted_class]

print(f"Predicted Classification: {predicted_label} (Class {predicted_class})")
print(f"Predicted Regression Value: {regression_prediction[0][0]}")
